In [8]:
import getpass
import os
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"


In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

result = model.invoke(messages)

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Ciao!'

In [21]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

In [12]:
system_command = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_command), ("user", "{text}")]
)
result = prompt_template.invoke({"language": "italian", "text": "hi"})
chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi"})


'ciao'

In [14]:
from langchain_core.messages import AIMessage

res = model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)
res.json()

'{"content": "Your name is Bob.", "additional_kwargs": {}, "response_metadata": {"token_usage": {"completion_tokens": 5, "prompt_tokens": 35, "total_tokens": 40}, "model_name": "gpt-4-0613", "system_fingerprint": null, "finish_reason": "stop", "logprobs": null}, "type": "ai", "name": null, "id": "run-004c25d5-2576-4deb-8614-4541398ce968-0", "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": {"input_tokens": 35, "output_tokens": 5, "total_tokens": 40}}'

In [15]:
## this is to store data in current session for the chatbot to remeber
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)
config = {"configurable": {"session_id": "abc2"}}

In [20]:
print(with_message_history.history_messages_key)

None


In [ ]:
print("Say Something ..")
while True:
    inp = input("Say something: ")
    response = with_message_history.invoke(
        [   
            HumanMessage(content=inp)
        ],
        config=config,
    )
    print()
    print(response.content)
    choice = input("Are you done with the chatbot? (y/n): ")
    if choice.lower() == "y":
        break


In [ ]:
def get_intent_scores(django_objects, user_query):
    
    prompt = """
    Task: You are given a list of Django model objects. Each object has a 'description' field. Your task is to
    1. Extract the 'description' field from each object.
    2. Understand the intent of the given user input query.
    3. Match the intent of the user query with each folder description.
    4. Return a dictionary where the keys are the 'name' values of the objects (treated as folder names) and the values are intent scores that indicate how well the user query matches each folder description
    Scoring should be on a scale from 0 to 1, where 1 means a perfect match and 0 means no match
    Example
    Given Django model objects:
    [
      {"id": 2,"name": "ProCare","description" :"This folder contains all the documents related to the Pro Care project." ,"parent_id": "1"},
      {"id": 3,"name": "Financials","description" : "This folder contains financial reports and documents related to the Pro Care project.","parent_id": "1"}
    ]

    User input query: "Where can I find the financial documents?
    Expected Output:
    {
      "Reports": 0.5,
      "Invoices": 0.9,
      "Receipts": 0.7
    }
    Now, given the following Django model objects and user input query, provide the required dictionary
    Django model objects:
    {django_objects}
    """
    
    template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                 prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    chain = template | model
    response = chain.invoke(
        {"messages": [HumanMessage(content=user_query)], "django_objects": django_objects}
    )

    return response.content


In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

In [29]:
prompt = f'''Task: You are given a list of Django model objects. Each object has a 'name' field. Your task is to extract the 'name' field from each object and create a description for each. Then, return a dictionary where the keys are the 'name' values (treated as folder names) and the values are the descriptions you create.

Example:

Given Django model objects:
[
  {"id": 2,"name": "Reports","parent_id": "1"},
  {"id": 3,"name": "Financials","created_at": "1"}
]

Expected Output:
{
  "Financial": "This folder contains all the reports related to the company and its financials.",
  "Invoices": "This folder contains files related to the information related to the expenses company made in year (any)",
  "Design Elements": "The folder contains the design elements realted to (certain) functionality of company.
}

Now, given the following Django model objects, provide the required dictionary: {objects}
'''

In [ ]:
from chat.models import File
messages = [
    SystemMessage(content="I have a django model named 'Folder' that has two fields, one is 'name' of type string and other is 'parent',that is a foreign key containing parent older of type self. I am providing you a list of objects of same type. What you have to do is extract 'name' field from those objects and create small description of about say 10 words using the folder name. Please try to understand the context of folder and then create descriptions. And I want you to return a dictionary containing folder names as keys and the description you created as values. The list of objects is {objects}.")
]
objects = File.objects.all()
result = model.invoke()

In [ ]:
from chat.models import File
objects = File.objects.all()

In [ ]:
prompt = f"""
Task: You are given a list of Django model objects. Each object has a 'description' field. Your task is to
1. Extract the 'description' field from each object.
2. Understand the intent of the given user input query.
3. Match the intent of the user query with each folder description.
4. Return a dictionary where the keys are the 'name' values of the objects (treated as folder names) and the values are intent scores that indicate how well the user query matches each folder description
Scoring should be on a scale from 0 to 1, where 1 means a perfect match and 0 means no match
Example
Given Django model objects:
[
  {"id": 2,"name": "Reports","parent_id": "1"},
  {"id": 3,"name": "Financials","created_at": "1"}
]

User input query: "Where can I find the financial documents?
Expected Output:
{
  "Reports": 0.5,
  "Invoices": 0.9,
  "Receipts": 0.7
}
Now, given the following Django model objects and user input query, provide the required dictionary
Django model objects:
{django_objects}
User input query: "{user_query}"
"""